### Logistic Model for Binary Classification

A retail banking institution is going to float a stock trading facility for their existing customer. Since this kind of facitlity is nothing new , company knows that they will have to incetivise their customers for adopting their offerings. One way to incetiwise is to offer discounts on the commision for trading transactions.

One issue with that is that only about 10% of the customers do enought trades for earnings after discounts to be profitable. Company wants to figure out, which are those 10% customer so that it can selectively offer them discount. there is no magic way to figure that out. So company rolled out this service to about 10000+ of their customers and observed their trading behaviour for 6 months and after that they labelled them into two revenue.grids 1 and 2.
using this data, now they want us to build a classification model which can be used to classify their remaining customers into these revenue grids.



#### Logistic Regression from Scikit Learn 

Logistic Regression in scikit learn already contains penalties. l1 and l2 [Read as L-one & L-two] penalties . l1 penalty is same as lasso penalty where as l2 is same as ridge penalty. parameter C for logistic regression function is the hyperparameter for penalty . However it works in inverse fashion, i.e. if C takes smalle , it means higher penalty. 

For the case that we have discussed here , we have discussed l1 penalty with value of C as 1. We have left following things for you to try on your own.

* model with l2 penalty
* Finding optimal value of hyperparameter C with cross-validation for both the penalties

You will find these in the practice exercise as well

you can use auc value obtained from function roc_auc_score to select best value for the hyperparameter. Higher the auc, better is the model . If you dont recall this, please go back to the theoretical reading material.

Lets beging our model building process.


In [1]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold
%matplotlib inline


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In C:\Users\s348a\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\s348a\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\s348a\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\s348a\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\s348a\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
data_file=r'Existing Base.csv'
bd=pd.read_csv(data_file)

In [3]:
bd.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


In [4]:
bd.columns

Index(['REF_NO', 'children', 'age_band', 'status', 'occupation',
       'occupation_partner', 'home_status', 'family_income', 'self_employed',
       'self_employed_partner', 'year_last_moved', 'TVarea', 'post_code',
       'post_area', 'Average Credit Card Transaction', 'Balance Transfer',
       'Term Deposit', 'Life Insurance', 'Medical Insurance',
       'Average A/C Balance', 'Personal Loan', 'Investment in Mutual Fund',
       'Investment Tax Saving Bond', 'Home Loan', 'Online Purchase Amount',
       'Revenue Grid', 'gender', 'region', 'Investment in Commudity',
       'Investment in Equity', 'Investment in Derivative',
       'Portfolio Balance'],
      dtype='object')

In [5]:
bd.dtypes

REF_NO                               int64
children                            object
age_band                            object
status                              object
occupation                          object
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan  

In [6]:
bd["children"].value_counts()

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

It seems we can directly convert this to numeric.

In [7]:
bd.loc[bd["children"]=="Zero","children"]="0"
bd.loc[bd["children"]=="4+","children"]="4"
bd["children"]=pd.to_numeric(bd["children"],errors="coerce")

In [8]:
bd["children"].value_counts()

0    6208
1    1848
2    1607
3     473
4      19
Name: children, dtype: int64

In [9]:
bd["Revenue Grid"].value_counts()

2    9069
1    1086
Name: Revenue Grid, dtype: int64

In [10]:
bd.columns

Index(['REF_NO', 'children', 'age_band', 'status', 'occupation',
       'occupation_partner', 'home_status', 'family_income', 'self_employed',
       'self_employed_partner', 'year_last_moved', 'TVarea', 'post_code',
       'post_area', 'Average Credit Card Transaction', 'Balance Transfer',
       'Term Deposit', 'Life Insurance', 'Medical Insurance',
       'Average A/C Balance', 'Personal Loan', 'Investment in Mutual Fund',
       'Investment Tax Saving Bond', 'Home Loan', 'Online Purchase Amount',
       'Revenue Grid', 'gender', 'region', 'Investment in Commudity',
       'Investment in Equity', 'Investment in Derivative',
       'Portfolio Balance'],
      dtype='object')

In [11]:
bd["y"]=np.where(bd["Revenue Grid"]==2,0,1)
bd=bd.drop(["Revenue Grid"],1)

In [12]:
bd['y'].value_counts()

0    9069
1    1086
Name: y, dtype: int64

For variable , age_band if we treat it as categorical variable , we can combine its categories by looking average response rate across its categories.

In [13]:
bd['age_band'].value_counts()

45-50      1359
36-40      1134
41-45      1112
31-35      1061
51-55      1052
55-60      1047
26-30       927
61-65       881
65-70       598
22-25       456
71+         410
18-21        63
Unknown      55
Name: age_band, dtype: int64

In [14]:
round(bd.groupby("age_band")["y"].mean(),2)

age_band
18-21      0.17
22-25      0.11
26-30      0.11
31-35      0.11
36-40      0.13
41-45      0.11
45-50      0.10
51-55      0.10
55-60      0.11
61-65      0.09
65-70      0.10
71+        0.10
Unknown    0.05
Name: y, dtype: float64

In [15]:
for i in range(len(bd)):
    if bd["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        bd.loc[i,"age_band"]="ab_10"
    if bd["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        bd.loc[i,"age_band"]="ab_11"
    if bd["age_band"][i]=="36-40":
        bd.loc[i,"age_band"]="ab_13"
    if bd["age_band"][i]=="18-21":
        bd.loc[i,"age_band"]="ab_17"
    if bd["age_band"][i]=="61-65":
        bd.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(bd["age_band"])
ab_dummies.head()

,Unknown,ab_10,ab_11,ab_13,ab_17,ab_9
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0


We will add it back to the dataset, dropping the dummy for "Unknown".

In [16]:
bd=pd.concat([bd,ab_dummies],1)
bd=bd.drop(["age_band","Unknown"],1)

In [17]:
bd.dtypes

REF_NO                               int64
children                             int64
status                              object
occupation                          object
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purc

In [18]:
bd["status"].value_counts()

Partner                 7709
Single/Never Married    1101
Divorced/Separated       679
Widowed                  618
Unknown                   48
Name: status, dtype: int64

In [19]:
bd["st_partner"]=np.where(bd["status"]=="Partner",1,0)
bd["st_singleNm"]=np.where(bd["status"]=="Single/Never Married",1,0)
bd["st_divSep"]=np.where(bd["status"]=="Divorced/Separated",1,0)
bd=bd.drop(["status"],1)

In [20]:
bd.dtypes

REF_NO                               int64
children                             int64
occupation                          object
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
gender     

In [21]:
bd['occupation'].value_counts()

Professional         2449
Retired              2206
Secretarial/Admin    1797
Housewife            1255
Business Manager      732
Unknown               567
Manual Worker         556
Other                 537
Student                56
Name: occupation, dtype: int64

In [22]:
round(bd.groupby("occupation")["y"].mean(),2)

occupation
Business Manager     0.12
Housewife            0.09
Manual Worker        0.11
Other                0.11
Professional         0.12
Retired              0.10
Secretarial/Admin    0.11
Student              0.11
Unknown              0.11
Name: y, dtype: float64

In [23]:
for i in range(len(bd)):
    if bd["occupation"][i] in ["Unknown","Student","Secretarial/Admin","Other","Manual Worker"]:
        bd.loc[i,"occupation"]="oc_11"
    if bd["occupation"][i] in ["Professional","Business Manager"]:
        bd.loc[i,"occupation"]="oc_12"
    if bd["occupation"][i]=="Retired":
        bd.loc[i,"occupation"]="oc_10"
oc_dummies=pd.get_dummies(bd["occupation"])
oc_dummies.head()

,Housewife,oc_10,oc_11,oc_12
0,0,0,1,0
1,0,1,0,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [24]:
bd=pd.concat([bd,oc_dummies],1)

bd=bd.drop(["occupation","Housewife"],1)

In [25]:
bd.dtypes

REF_NO                               int64
children                             int64
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
gender                              object
region     

In [26]:
round(bd.groupby("occupation_partner")["y"].mean(),2)

occupation_partner
Business Manager     0.11
Housewife            0.11
Manual Worker        0.11
Other                0.10
Professional         0.11
Retired              0.10
Secretarial/Admin    0.12
Student              0.12
Unknown              0.10
Name: y, dtype: float64

In [27]:
bd['TVarea'].value_counts()

Central          1618
Carlton          1541
Meridian         1226
Yorkshire        1042
Granada          1039
HTV               845
Anglia            751
Tyne Tees         536
Scottish TV       520
TV South West     363
Grampian          227
Unknown           189
Ulster            159
Border             99
Name: TVarea, dtype: int64

In [28]:
round(bd.groupby("TVarea")["y"].mean(),2)

TVarea
Anglia           0.11
Border           0.14
Carlton          0.11
Central          0.11
Grampian         0.10
Granada          0.12
HTV              0.10
Meridian         0.09
Scottish TV      0.11
TV South West    0.09
Tyne Tees        0.09
Ulster           0.12
Unknown          0.12
Yorkshire        0.12
Name: y, dtype: float64

In [29]:
round(bd.groupby("post_area")["y"].mean(),2)

post_area
AB12    0.00
AB13    0.00
AB14    0.25
AB15    0.00
AB21    0.25
        ... 
YO62    1.00
YO7     0.00
YO8     0.00
ZE1     0.00
ZE2     0.00
Name: y, Length: 2039, dtype: float64

In [30]:
round(bd.groupby("region")["y"].mean(),2)

region
Channel Islands     0.20
East Anglia         0.13
East Midlands       0.10
Isle of Man         0.00
North               0.10
North West          0.11
Northern Ireland    0.12
Scotland            0.11
South East          0.10
South West          0.09
Unknown             0.12
Wales               0.11
West Midlands       0.11
Name: y, dtype: float64

In [31]:
bd["ocp_10"]=0
bd["ocp_12"]=0
for i in range(len(bd)):
    if bd["occupation_partner"][i] in ["Unknown","Retired","Other"]:
        bd.loc[i,"ocp_10"]=1
    if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
        bd.loc[i,"ocp_12"]=1
        
bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)

In [32]:
bd.dtypes

REF_NO                               int64
children                             int64
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
gender                              object
Investment in Commudity            float64
Investment in Equity               float64
Investment in Derivative           float64
Portfolio Balance                  float64
y          

You can see that we have also dropped variables TVarea, region, post_code, post_area. If you look at number of unique values taken by post_area and post_code , you'll realise why decided to drop them. TVarea and region on the other hand we have left for you to make use of and see if using them improves your model.

In [33]:
bd["home_status"].value_counts()

Own Home                9413
Rent from Council/HA     322
Rent Privately           261
Live in Parental Hom     109
Unclassified              50
Name: home_status, dtype: int64

In [34]:
bd["hs_own"]=np.where(bd["home_status"]=="Own Home",1,0)
del bd["home_status"]

Notice that we used an alternate syntax for dropping a column here. You can use that too if you like this syntax better.

In [35]:
bd["gender"].value_counts()

Female     7634
Male       2486
Unknown      35
Name: gender, dtype: int64

In [36]:
bd["gender_f"]=np.where(bd["gender"]=="Female",1,0)
del bd["gender"]

In [37]:
bd["self_employed"].value_counts()

No     9436
Yes     719
Name: self_employed, dtype: int64

In [38]:
bd["semp_yes"]=np.where(bd["self_employed"]=="Yes",1,0)
del bd["self_employed"]

In [39]:
bd["self_employed_partner"].value_counts()

No     9026
Yes    1129
Name: self_employed_partner, dtype: int64

In [40]:
bd["semp_part_yes"]=np.where(bd["self_employed_partner"]=="Yes",1,0)
del bd["self_employed_partner"]

In [41]:
bd.dtypes

REF_NO                               int64
children                             int64
family_income                       object
year_last_moved                      int64
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
Investment in Commudity            float64
Investment in Equity               float64
Investment in Derivative           float64
Portfolio Balance                  float64
y                                    int32
ab_10                                uint8
ab_11                                uint8
ab_13                                uint8
ab_17      

In [42]:
bd["family_income"].value_counts()

>=35,000             2517
<27,500, >=25,000    1227
<30,000, >=27,500     994
<25,000, >=22,500     833
<20,000, >=17,500     683
<12,500, >=10,000     677
<17,500, >=15,000     634
<15,000, >=12,500     629
<22,500, >=20,000     590
<10,000, >= 8,000     563
< 8,000, >= 4,000     402
< 4,000               278
Unknown               128
Name: family_income, dtype: int64

We can convert this to number as average of the range once we have figured out what to do with category "Unknown".

In [43]:
round(bd.groupby("family_income")["y"].mean(),4)

family_income
< 4,000              0.0755
< 8,000, >= 4,000    0.0796
<10,000, >= 8,000    0.1066
<12,500, >=10,000    0.1019
<15,000, >=12,500    0.1113
<17,500, >=15,000    0.1230
<20,000, >=17,500    0.1113
<22,500, >=20,000    0.1186
<25,000, >=22,500    0.1032
<27,500, >=25,000    0.0970
<30,000, >=27,500    0.1157
>=35,000             0.1116
Unknown              0.0703
Name: y, dtype: float64

In [44]:
bd["fi"]=4 # by doing this , we have essentially clubbed <4000 and Unknown values . How?
bd.loc[bd["family_income"]=="< 8,000, >= 4,000","fi"]=6
bd.loc[bd["family_income"]=="<10,000, >= 8,000","fi"]=9
bd.loc[bd["family_income"]=="<12,500, >=10,000","fi"]=11.25
bd.loc[bd["family_income"]=="<15,000, >=12,500","fi"]=13.75
bd.loc[bd["family_income"]=="<17,500, >=15,000","fi"]=16.25
bd.loc[bd["family_income"]=="<20,000, >=17,500","fi"]=18.75
bd.loc[bd["family_income"]=="<22,500, >=20,000","fi"]=21.25
bd.loc[bd["family_income"]=="<25,000, >=22,500","fi"]=23.75
bd.loc[bd["family_income"]=="<27,500, >=25,000","fi"]=26.25
bd.loc[bd["family_income"]=="<30,000, >=27,500","fi"]=28.75
bd.loc[bd["family_income"]==">=35,000","fi"]=35
bd=bd.drop(["family_income"],1)

In [45]:
bd.dtypes

REF_NO                               int64
children                             int64
year_last_moved                      int64
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
Investment in Commudity            float64
Investment in Equity               float64
Investment in Derivative           float64
Portfolio Balance                  float64
y                                    int32
ab_10                                uint8
ab_11                                uint8
ab_13                                uint8
ab_17                                uint8
ab_9       

In [46]:
bd.head()

,REF_NO,children,year_last_moved,Average Credit Card Transaction,Balance Transfer,Term Deposit,Life Insurance,Medical Insurance,Average A/C Balance,Personal Loan,...,oc_10,oc_11,oc_12,ocp_10,ocp_12,hs_own,gender_f,semp_yes,semp_part_yes,fi
0,1,0,1972,148.44,142.95,0.00,81.96,0.00,29.99,0.00,...,0,1,0,0,1,1,1,0,0,16.25
1,2,0,1998,0.00,74.98,0.00,25.99,0.00,0.00,0.00,...,1,0,0,1,0,1,1,0,0,26.25
2,3,0,1996,0.00,166.44,20.99,291.37,11.48,166.94,0.00,...,0,0,1,1,0,1,0,1,0,28.75
3,5,0,1997,0.00,0.00,0.00,20.49,0.00,39.46,0.00,...,0,0,1,0,0,1,1,0,0,13.75
4,6,0,1995,73.45,57.96,0.00,177.42,41.95,39.47,10.97,...,0,0,1,1,0,1,1,0,0,28.75


In [47]:
bd.isnull().sum()

REF_NO                             0
children                           0
year_last_moved                    0
Average Credit Card Transaction    0
Balance Transfer                   0
Term Deposit                       0
Life Insurance                     0
Medical Insurance                  0
Average A/C Balance                0
Personal Loan                      0
Investment in Mutual Fund          0
Investment Tax Saving Bond         0
Home Loan                          0
Online Purchase Amount             0
Investment in Commudity            0
Investment in Equity               0
Investment in Derivative           0
Portfolio Balance                  0
y                                  0
ab_10                              0
ab_11                              0
ab_13                              0
ab_17                              0
ab_9                               0
st_partner                         0
st_singleNm                        0
st_divSep                          0
o

Now that the entire data is of numeric type, lets beging our modelling process after removing nas from the data.

In [48]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to t

In [49]:
bd.dropna(axis=0,inplace=True)
bd_train, bd_test = train_test_split(bd, test_size = 0.2,random_state=2)

In [50]:
bd.shape

(10155, 37)

In [51]:
bd_test.shape

(2031, 37)

In [52]:
bd_train['y'].value_counts()

0    7262
1     862
Name: y, dtype: int64

In [53]:
bd_test['y'].value_counts()

0    1807
1     224
Name: y, dtype: int64

In [54]:
x_train=bd_train.drop(["y","REF_NO"],1)
y_train=bd_train["y"]
x_test=bd_test.drop(["y","REF_NO"],1)
y_test=bd_test["y"]

In [55]:
x_train.head()

,children,year_last_moved,Average Credit Card Transaction,Balance Transfer,Term Deposit,Life Insurance,Medical Insurance,Average A/C Balance,Personal Loan,Investment in Mutual Fund,...,oc_10,oc_11,oc_12,ocp_10,ocp_12,hs_own,gender_f,semp_yes,semp_part_yes,fi
2834,0,1982,0.00,34.49,0.00,0.00,0.00,38.94,0.0,11.99,...,0,1,0,0,0,1,1,0,0,35.00
4036,2,1980,0.00,0.00,0.00,0.00,54.95,0.00,0.0,216.88,...,0,1,0,0,0,1,1,0,0,35.00
6288,0,1979,0.00,79.97,0.00,19.99,0.00,82.39,0.0,20.98,...,1,0,0,1,0,1,1,0,0,6.00
10120,0,1998,0.00,17.99,62.46,133.93,11.48,0.00,0.0,0.00,...,0,0,1,1,0,1,0,0,0,16.25
3286,2,1983,15.99,49.98,49.47,14.99,24.27,19.99,0.0,19.48,...,0,0,0,0,0,1,1,0,1,35.00


In [56]:
y_train.head()

2834     0
4036     0
6288     0
10120    0
3286     0
Name: y, dtype: int32

In [57]:
# Logistic regression
# KNN
# Support Vector Machine
# Decision Tree
# Random Forest 
# xgboost

In [58]:
help(LogisticRegression)

Help on class LogisticRegression in module sklearn.linear_model._logistic:

class LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', 's

In [59]:
logr=LogisticRegression(penalty="l2",class_weight="balanced",random_state=2)

In [60]:
logr.fit(x_train,y_train)

C:\Users\s348a\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced', random_state=2)

In [62]:
# score model performance on the test data
roc_auc_score(y_test,logr.predict(x_test))

0.8912537552375683

To arrive at the eventual 1,0 prediction , we need to find some way [some cutoff ] to convert predicted probabilites into two classes. Lets first get the probabilities out.

In [64]:
prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1])
prob_score

0       0.048503
1       0.056762
2       0.018438
3       0.149126
4       0.035209
          ...   
8119    0.005200
8120    0.002637
8121    0.078086
8122    0.748631
8123    1.000000
Length: 8124, dtype: float64

On these scores , we will consider many cutoffs between 0 to 1

In [67]:
cutoffs=np.linspace(0,1,100)
cutoffs

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

For each of these cutoff , we are going to look at TP,FP,TN,FN values and caluclate KS. Then we'll chose the best cutoff as the one having highest KS.

In [68]:
KS_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])
    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]

In [70]:
KS_cutoff

44    0.444444
Name: cutoff, dtype: float64

Now we'll see how this model with the cutoff determined here , performs on the test data.

In [71]:
# Performance on test data
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

confusion matrix :
 
  predicted     0    1
real                
0          1635  172
1            26  198


In [72]:
# Accuracy of test
(TP+TN)/(P+N)

0.9025110782865583

In [73]:
# Sensitivity on test
TP/P

0.8839285714285714

In [74]:
#Specificity on test
TN/N

0.904814609850581

Next we see how cutoff determined by F_beta score performs on test data for beta values : 0.5,1,2

In [75]:
cutoffs=np.linspace(0.010,0.99,100)
def Fbeta_perf(beta,cutoffs,y_train,prob_score):
    FB_cut=[]
    for cutoff in cutoffs:
        predicted=pd.Series([0]*len(y_train))
        predicted[prob_score>cutoff]=1
        df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        
        
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_cut.append(FB)

    cutoff_data=pd.DataFrame(list(zip(cutoffs,FB_cut)),columns=["cutoff","FB"])

    FB_cutoff=cutoff_data[cutoff_data["FB"]==cutoff_data["FB"].max()]["cutoff"]

    prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

    predicted_test=pd.Series([0]*len(y_test))
    predicted_test[prob_score_test>float(FB_cutoff)]=1

    df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

    k=pd.crosstab(df_test['real'],df_test["predicted"])
#     print('confusion matrix :\n \n ',k)
    TN=k.iloc[0,0]
    TP=k.iloc[1,1]
    FP=k.iloc[0,1]
    FN=k.iloc[1,0]
    P=TP+FN
    N=TN+FP
    print('For beta :',beta)
    print('Accuracy is :',(TP+TN)/(P+N))
    print('Sensitivity is :',(TP/P))
    print('Specificity is :',(TN/N))
    print('\n \n \n')

In [76]:
Fbeta_perf(0.5,cutoffs,y_train,prob_score)
Fbeta_perf(1,cutoffs,y_train,prob_score)
Fbeta_perf(2,cutoffs,y_train,prob_score)

For beta : 0.5
Accuracy is : 0.9345150172328902
Sensitivity is : 0.5044642857142857
Specificity is : 0.987825124515772

 
 

For beta : 1
Accuracy is : 0.9290989660265879
Sensitivity is : 0.7366071428571429
Specificity is : 0.9529607083563918

 
 

For beta : 2
Accuracy is : 0.9162973904480551
Sensitivity is : 0.8526785714285714
Specificity is : 0.9241837299391257

 
 



You can see that low beta < 1 favors Specificity where as beta > 1 favors sensitivity